In [5]:
import pandas as pd 
import numpy as np

In [10]:
df = pd.read_csv('nuclei-coordinates.csv', header=None)
coordinates = df.values

In [13]:
print(len(coordinates))

3340


We can do several things from here with our 3000 pair dataset